In [1]:
from pyspark.sql import SparkSession
import os
import re

# Aplinkos paruošimas
os.environ["PYSPARK_PYTHON"] = r"C:\Users\Eivinas\DDA\ld2\venv\Scripts\python.exe"
os.environ["PYSPARK_DRIVER_PYTHON"] = r"C:\Users\Eivinas\DDA\ld2\venv\Scripts\python.exe"

In [2]:
spark = SparkSession.builder \
    .appName("DDA LD2") \
    .master("local[*]") \
    .config("spark.driver.memory", "4g") \
    .config("spark.local.dir", "C:/tmp/spark-temp") \
    .getOrCreate()

sc = spark.sparkContext

# Patikriname ar veikia
spark

In [3]:
# Funkcija skirta ištraukti įrašus iš teksto eilutės
def parse_line(line):
    entries = line.split("{{")[1:]
    parsed_entries = []
    for entry in entries:
        params = {param.split("=")[0]:param.split("=")[1].strip("}") for param in entry.split("}{")}
        parsed_entries.append(params)
    return parsed_entries

# Funkcija skirta ištraukti konkrečius laukus iš įrašo
# Čia:
# - entry - įrašas
# - group_by_fields - laukai, pagal kuriuos grupuojame
# - value_fields - laukai, kuriuos grupuosime pagal group_by_fields
# - value_casts - duomenų tipai, į kuriuos konvertuosime value_fields
# pvz. value_casts = [int, float] konvertuos value_fields[0] į int ir value_fields[1] į float
def extract(entry, group_by_fields, value_fields=None, value_casts=None):
    if value_casts:
        assert len(value_fields) == len(value_casts), "value_fields and value_casts must have the same length"
    try:
        # Gauti grupavimo laukus
        group_key = tuple(entry[f] for f in group_by_fields)
        if len(group_key) == 1:
            group_key = group_key[0]

        if value_fields is None:
            return group_key  # Jei nėra value_fields, grąžiname tik grupavimo laukus

        values = []
        for i, field in enumerate(value_fields):
            val = entry[field]
            if value_casts:
                val = value_casts[i](val)
            values.append(val)

        return (group_key, tuple(values))
    except (KeyError, ValueError):
        return None


# Įkrauti ir apdoroti duomenis
rdd = sc.textFile("duom_full.txt")
entries_rdd = rdd.flatMap(parse_line)

In [4]:
# Pirmas uždavinys

# sg - svoris, grupe
sg_rdd = entries_rdd.map(
    lambda entry: extract(
        entry,
        group_by_fields=['svorio grupe'],
        value_fields=['svoris'],
        value_casts=[float]
    )
).filter(lambda x: x is not None).mapValues(lambda x: (x[0], 1, x[0], x[0]))

reduced_rdd = sg_rdd.reduceByKey(
    lambda a, b: (a[0] + b[0], a[1] + b[1], min(a[2], b[2]), max(a[3], b[3]))
)

stats_rdd = reduced_rdd.mapValues(lambda v: {
    'avg': v[0] / v[1],
    'min': v[2],
    'max': v[3]
})

stats_results = stats_rdd.collect()

print(f"{'Grupe':<10}{'Min':>10}{'Max':>10}{'Vid':>10}")
print("-" * 40)
for group, stats in stats_results:
    print(f"{group:<10}{stats['min']:>10.2f}{stats['max']:>10.2f}{stats['avg']:>10.2f}")

Grupe            Min       Max       Vid
----------------------------------------
<50             0.00     50.00      5.96
>300          300.05   6896.65    759.21
<300           50.05    300.00    110.03


In [5]:
# mz - maršrutas, zona
mz_rdd = entries_rdd.map(
    lambda entry: extract(
        entry,
        group_by_fields=['marsrutas'],
        value_fields=['geografine zona']
    )
).filter(lambda x: x is not None)

# Sugrupuojame pagal maršrutą ir surenkame zonas
zones_by_route = mz_rdd.groupByKey().mapValues(lambda zones: set(zones))

# Filtruojame maršrutus, kurie aplanko daugiau nei vieną zoną
multi_zone_routes = zones_by_route.filter(lambda x: len(x[1]) > 1)

# Suskaičiuojame procentą
all_routes_count = zones_by_route.count()
multi_zone_route_count = multi_zone_routes.count()
multi_zone_percentage = (multi_zone_route_count / all_routes_count) * 100 if all_routes_count else 0

print(f"Iš viso: {multi_zone_route_count}/{all_routes_count} maršrutai | Procentas: {multi_zone_percentage:.2f}%\n")

Iš viso: 398/424 maršrutai | Procentas: 93.87%



In [6]:
# mdz - marsrutas, diena, zona
mdz_rdd = entries_rdd.map(
    lambda entry: extract(
        entry,
        group_by_fields=['marsrutas', 'sustojimo data'],
        value_fields=['geografine zona']
    )
).filter(lambda x: x is not None)

# Sugrupuojame pagal maršrutą ir datą, surenkame zonas
zones_by_route_date = mdz_rdd.groupByKey().mapValues(lambda zones: set(zones))

# Filtruojame maršrutus, kurie aplanko daugiau nei vieną zoną tą pačią dieną
multi_zone_same_day = zones_by_route_date.filter(lambda x: len(x[1]) > 1)

# Suskaičiuojame procentą
multi_same_day_route_count = multi_zone_same_day.map(lambda x: x[0][0]).distinct().count()
multi_same_day_percentage = (multi_same_day_route_count / all_routes_count) * 100

print(f"Iš viso: {multi_same_day_route_count}/{all_routes_count} maršrutai | Procentas: {multi_same_day_percentage:.2f}%")

Iš viso: 396/424 maršrutai | Procentas: 93.40%


In [7]:
# skzd - siuntos, klientai, zonos, dienos
skzd_rdd = entries_rdd.map(
    lambda entry: extract(
        entry,
        group_by_fields=['geografine zona', 'sustojimo savaites diena'],
        value_fields=['siuntu skaicius', 'Sustojimo klientu skaicius'],
        value_casts=[int, int]
    )
).filter(lambda x: x is not None)

summed_skzd_rdd = skzd_rdd.reduceByKey(
    lambda a, b: (a[0] + b[0], a[1] + b[1])
)

skzd_results = summed_skzd_rdd.collect()

skzd_results.sort(key=lambda x: (x[0], x[1]))

print(f"{'Zona':<10}{'Diena':>5}{'Siuntu skaicius':>20}{'Klientu skaicius':>20}")
print("-" * 55)
for (zone, weekday), (siuntos_sum, klientai_sum) in skzd_results:
    print(f"{zone:<10}{weekday:>5}{siuntos_sum:>20}{klientai_sum:>20}")

Zona      Diena     Siuntu skaicius    Klientu skaicius
-------------------------------------------------------
              1                  19                  16
              2                  13                  11
              3                  15                  12
              4                   7                   8
              5                  10                   8
Z1            1               86464               41580
Z1            2              123938               60167
Z1            3              126872               62241
Z1            4               98156               48430
Z1            5               88663               45999
Z1            6                1754                 797
Z2            1               19525               11751
Z2            2               31377               18499
Z2            3               30715               18545
Z2            4               23727               14190
Z2            5               23883             